# Тестирование моделей актора и критика

Этот ноутбук позволяет загружать обученные модели актора и критика по имени эксперимента и тестировать их на фиксированных входных данных.


In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from typing import Tuple, Optional, Dict, Any
from omegaconf import DictConfig, OmegaConf

from nn_laser_stabilizer.agents.td3 import make_td3_agent, warmup_from_specs
from nn_laser_stabilizer.envs.utils import make_specs


## Функции для загрузки моделей


In [ ]:
def find_experiment_path(experiment_name: str, date: str, time: str) -> Path:
    """
    Находит путь к эксперименту по имени, дате и времени.
    
    Args:
        experiment_name: Имя эксперимента (например, 'td3_train_simulation')
        date: Дата в формате YYYY-MM-DD (опционально)
        time: Время в формате HH-MM-SS (опционально)
    
    Returns:
        Path к директории эксперимента
    """
    experiments_dir = Path("../experiments")
    
    if not experiments_dir.exists():
        raise FileNotFoundError(f"Директория экспериментов не найдена: {experiments_dir}")
    
    exp_dir = experiments_dir / experiment_name
    
    if not exp_dir.exists():
        raise FileNotFoundError(f"Эксперимент не найден: {experiment_name}")
    
    date_dir = exp_dir / date
    
    if not date_dir.exists():
        raise FileNotFoundError(f"Дата не найдена: {date}")
    
    time_dir = date_dir / time
    
    if not time_dir.exists():
        raise FileNotFoundError(f"Время не найдено: {time}")
    
    return time_dir

def load_experiment_config(experiment_path: Path) -> DictConfig:
    """
    Загружает конфигурацию эксперимента.
    
    Args:
        experiment_path: Путь к директории эксперимента
    
    Returns:
        Конфигурация эксперимента
    """
    config_path = experiment_path / ".hydra" / "config.yaml"
    
    if config_path.exists():
        return OmegaConf.load(config_path)
    else:
        raise FileNotFoundError(f"Файл конфигурации не найден: {config_path}")

def load_models(experiment_name: str, date: Optional[str] = None, time: Optional[str] = None, 
                device: str = "cpu") -> Tuple[torch.nn.Module, torch.nn.Module, DictConfig]:
    """
    Загружает модели актора и критика из эксперимента.
    
    Args:
        experiment_name: Имя эксперимента
        date: Дата эксперимента (опционально)
        time: Время эксперимента (опционально)
        device: Устройство для загрузки моделей
    
    Returns:
        Кортеж (actor, critic, config)
    """
    # Находим путь к эксперименту
    experiment_path = find_experiment_path(experiment_name, date, time)
    print(f"Загружаем модели из: {experiment_path}")
    
    # Загружаем конфигурацию
    config = load_experiment_config(experiment_path)
    print(f"Конфигурация загружена: {config.experiment_name}")
    
    # Получаем спецификации из конфигурации
    specs = make_specs(config.env.bounds)
    action_spec = specs["action"]
    observation_spec = specs["observation"]
    
    # Создаем модели
    actor, critic = make_td3_agent(config, observation_spec, action_spec)
    
    # Загружаем веса моделей
    saved_models_dir = experiment_path / "saved_models"
    
    if saved_models_dir.exists():
        actor_path = saved_models_dir / "actor.pth"
        critic_path = saved_models_dir / "qvalue.pth"
        
        if actor_path.exists():
            actor.load_state_dict(torch.load(actor_path, map_location=device))
            print(f"Актор загружен из: {actor_path}")
        else:
            print(f"Файл актора не найден: {actor_path}")
        
        if critic_path.exists():
            critic.load_state_dict(torch.load(critic_path, map_location=device))
            print(f"Критик загружен из: {critic_path}")
        else:
            print(f"Файл критика не найден: {critic_path}")
    else:
        print(f"Директория с сохраненными моделями не найдена: {saved_models_dir}")
        print("Используются случайно инициализированные веса")
    
    # Переводим модели на нужное устройство
    actor = actor.to(device)
    critic = critic.to(device)
    
    # Выполняем warmup
    warmup_from_specs(observation_spec, action_spec, actor, critic, device)
    
    return actor, critic, config


## Функции для тестирования моделей


In [ ]:
def test_actor(actor: torch.nn.Module, observations: torch.Tensor, device: str = "cpu") -> torch.Tensor:
    """
    Тестирует актора на заданных наблюдениях.
    
    Args:
        actor: Модель актора
        observations: Тензор наблюдений [batch_size, obs_dim]
        device: Устройство для вычислений
    
    Returns:
        Тензор действий [batch_size, action_dim]
    """
    actor.eval()
    
    with torch.no_grad():
        # Создаем TensorDict для актора
        from tensordict import TensorDict
        from torchrl.envs import set_exploration_type, ExplorationType
        
        batch_size = observations.shape[0]
        td = TensorDict({
            "observation": observations.to(device),
            "is_init": torch.ones(batch_size, dtype=torch.bool, device=device)
        }, batch_size=[batch_size])
        
        # Получаем действия в детерминированном режиме
        with set_exploration_type(ExplorationType.DETERMINISTIC):
            actions = actor(td)["action"]
    
    return actions

def test_critic(critic: torch.nn.Module, observations: torch.Tensor, actions: torch.Tensor, 
                device: str = "cpu") -> torch.Tensor:
    """
    Тестирует критика на заданных наблюдениях и действиях.
    
    Args:
        critic: Модель критика
        observations: Тензор наблюдений [batch_size, obs_dim]
        actions: Тензор действий [batch_size, action_dim]
        device: Устройство для вычислений
    
    Returns:
        Тензор Q-значений [batch_size, 1]
    """
    critic.eval()
    
    with torch.no_grad():
        # Создаем TensorDict для критика
        from tensordict import TensorDict
        
        batch_size = observations.shape[0]
        td = TensorDict({
            "observation": observations.to(device),
            "action": actions.to(device),
            "is_init": torch.ones(batch_size, dtype=torch.bool, device=device)
        }, batch_size=[batch_size])
        
        # Получаем Q-значения
        q_values = critic(td)["state_action_value"]
    
    return q_values

def generate_test_data(observation_spec, action_spec, num_samples: int = 10, 
                      device: str = "cpu") -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Генерирует тестовые данные для моделей.
    
    Args:
        observation_spec: Спецификация наблюдений
        action_spec: Спецификация действий
        num_samples: Количество тестовых образцов
        device: Устройство для тензоров
    
    Returns:
        Кортеж (observations, actions)
    """
    # Генерируем случайные наблюдения в пределах спецификации
    obs_shape = (num_samples,) + observation_spec.shape
    observations = torch.rand(obs_shape, device=device) * (observation_spec.space.high - observation_spec.space.low) + observation_spec.space.low
    
    # Генерируем случайные действия в пределах спецификации
    action_shape = (num_samples,) + action_spec.shape
    actions = torch.rand(action_shape, device=device) * (action_spec.space.high - action_spec.space.low) + action_spec.space.low
    
    return observations, actions

def test_models_comprehensive(actor: torch.nn.Module, critic: torch.nn.Module, 
                            observation_spec, action_spec, num_samples: int = 10,
                            device: str = "cpu") -> Dict[str, Any]:
    """
    Комплексное тестирование моделей актора и критика.
    
    Args:
        actor: Модель актора
        critic: Модель критика
        observation_spec: Спецификация наблюдений
        action_spec: Спецификация действий
        num_samples: Количество тестовых образцов
        device: Устройство для вычислений
    
    Returns:
        Словарь с результатами тестирования
    """
    print(f"Тестируем модели на {num_samples} образцах...")
    
    # Генерируем тестовые данные
    observations, random_actions = generate_test_data(observation_spec, action_spec, num_samples, device)
    
    # Тестируем актора
    actor_actions = test_actor(actor, observations, device)
    
    # Тестируем критика с действиями от актора
    critic_q_values_actor = test_critic(critic, observations, actor_actions, device)
    
    # Тестируем критика со случайными действиями
    critic_q_values_random = test_critic(critic, observations, random_actions, device)
    
    results = {
        "observations": observations,
        "actor_actions": actor_actions,
        "random_actions": random_actions,
        "critic_q_values_actor": critic_q_values_actor,
        "critic_q_values_random": critic_q_values_random,
        "num_samples": num_samples
    }
    
    # Выводим статистику
    print(f"\nСтатистика актора:")
    print(f"  Среднее действие: {actor_actions.mean(dim=0)}")
    print(f"  Стандартное отклонение: {actor_actions.std(dim=0)}")
    print(f"  Минимум: {actor_actions.min(dim=0)[0]}")
    print(f"  Максимум: {actor_actions.max(dim=0)[0]}")
    
    print(f"\nСтатистика критика (действия от актора):")
    print(f"  Среднее Q-значение: {critic_q_values_actor.mean():.4f}")
    print(f"  Стандартное отклонение: {critic_q_values_actor.std():.4f}")
    print(f"  Минимум: {critic_q_values_actor.min():.4f}")
    print(f"  Максимум: {critic_q_values_actor.max():.4f}")
    
    print(f"\nСтатистика критика (случайные действия):")
    print(f"  Среднее Q-значение: {critic_q_values_random.mean():.4f}")
    print(f"  Стандартное отклонение: {critic_q_values_random.std():.4f}")
    print(f"  Минимум: {critic_q_values_random.min():.4f}")
    print(f"  Максимум: {critic_q_values_random.max():.4f}")
    
    return results

## Функции для визуализации результатов


In [ ]:
def plot_test_results(results: Dict[str, Any], title: str = "Результаты тестирования моделей"):
    """
    Визуализирует результаты тестирования моделей.
    
    Args:
        results: Словарь с результатами тестирования
        title: Заголовок графика
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(title, fontsize=16)
    
    observations = results["observations"]
    actor_actions = results["actor_actions"]
    random_actions = results["random_actions"]
    critic_q_values_actor = results["critic_q_values_actor"]
    critic_q_values_random = results["critic_q_values_random"]
    
    # График 1: Действия актора
    axes[0, 0].plot(actor_actions.cpu().numpy())
    axes[0, 0].set_title("Действия актора")
    axes[0, 0].set_xlabel("Образец")
    axes[0, 0].set_ylabel("Значение действия")
    axes[0, 0].legend([f"Действие {i}" for i in range(actor_actions.shape[1])])
    axes[0, 0].grid(True)
    
    # График 2: Q-значения критика
    axes[0, 1].plot(critic_q_values_actor.cpu().numpy(), label="Действия от актора", alpha=0.7)
    axes[0, 1].plot(critic_q_values_random.cpu().numpy(), label="Случайные действия", alpha=0.7)
    axes[0, 1].set_title("Q-значения критика")
    axes[0, 1].set_xlabel("Образец")
    axes[0, 1].set_ylabel("Q-значение")
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # График 3: Распределение действий актора
    for i in range(actor_actions.shape[1]):
        axes[1, 0].hist(actor_actions[:, i].cpu().numpy(), alpha=0.7, label=f"Действие {i}")
    axes[1, 0].set_title("Распределение действий актора")
    axes[1, 0].set_xlabel("Значение действия")
    axes[1, 0].set_ylabel("Частота")
    axes[1, 0].legend()
    axes[1, 0].grid(True)
    
    # График 4: Сравнение Q-значений
    axes[1, 1].scatter(critic_q_values_actor.cpu().numpy(), critic_q_values_random.cpu().numpy(), alpha=0.7)
    axes[1, 1].plot([critic_q_values_actor.min(), critic_q_values_actor.max()], 
                    [critic_q_values_actor.min(), critic_q_values_actor.max()], 'r--', alpha=0.5)
    axes[1, 1].set_title("Сравнение Q-значений")
    axes[1, 1].set_xlabel("Q-значение (действия от актора)")
    axes[1, 1].set_ylabel("Q-значение (случайные действия)")
    axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.show()

def compare_experiments(experiment_results: Dict[str, Dict[str, Any]]):
    """
    Сравнивает результаты нескольких экспериментов.
    
    Args:
        experiment_results: Словарь с результатами экспериментов
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle("Сравнение экспериментов", fontsize=16)
    
    experiment_names = list(experiment_results.keys())
    colors = plt.cm.Set3(np.linspace(0, 1, len(experiment_names)))
    
    # График 1: Средние Q-значения для действий от актора
    mean_q_actor = []
    for name in experiment_names:
        mean_q_actor.append(experiment_results[name]["critic_q_values_actor"].mean().item())
    
    axes[0, 0].bar(experiment_names, mean_q_actor, color=colors)
    axes[0, 0].set_title("Средние Q-значения (действия от актора)")
    axes[0, 0].set_ylabel("Q-значение")
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # График 2: Средние Q-значения для случайных действий
    mean_q_random = []
    for name in experiment_names:
        mean_q_random.append(experiment_results[name]["critic_q_values_random"].mean().item())
    
    axes[0, 1].bar(experiment_names, mean_q_random, color=colors)
    axes[0, 1].set_title("Средние Q-значения (случайные действия)")
    axes[0, 1].set_ylabel("Q-значение")
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # График 3: Разброс действий актора
    action_std = []
    for name in experiment_names:
        action_std.append(experiment_results[name]["actor_actions"].std().mean().item())
    
    axes[1, 0].bar(experiment_names, action_std, color=colors)
    axes[1, 0].set_title("Средний разброс действий актора")
    axes[1, 0].set_ylabel("Стандартное отклонение")
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # График 4: Разность Q-значений (актор vs случайные)
    q_diff = []
    for name in experiment_names:
        diff = (experiment_results[name]["critic_q_values_actor"].mean() - 
                experiment_results[name]["critic_q_values_random"].mean()).item()
        q_diff.append(diff)
    
    axes[1, 1].bar(experiment_names, q_diff, color=colors)
    axes[1, 1].set_title("Разность Q-значений (актор - случайные)")
    axes[1, 1].set_ylabel("Разность Q-значений")
    axes[1, 1].tick_params(axis='x', rotation=45)
    axes[1, 1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.show()


## Тестирование конкретных экспериментов

Тестируем модели из экспериментов от 2025-09-24:


In [ ]:
# Параметры экспериментов
EXPERIMENT_DATE = "2025-09-24"
EXPERIMENTS = {
    "relative_skip1": {
        "time": "18-31-57",
    },
    "relative_skip50": {
        "time": "18-52-01",
    },
    "absolute_skip50": {
        "time": "18-14-49",
    },
}

# Устройство для вычислений
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Используемое устройство: {device}")

# Количество тестовых образцов
num_test_samples = 20


In [ ]:
# Загружаем и тестируем модели для каждого эксперимента
experiment_results = {}
loaded_models = {}

for exp_name, exp_info in EXPERIMENTS.items():
    print(f"\n{'='*60}")
    print(f"Тестируем эксперимент: {exp_name}")
    print(f"Дата: {EXPERIMENT_DATE}, Время: {exp_info['time']}")
    print(f"{'='*60}")
    
    try:
        # Загружаем модели
        actor, critic, config = load_models(
            experiment_name=exp_name,
            date=EXPERIMENT_DATE,
            time=exp_info['time'],
            device=device
        )
        
        # Сохраняем загруженные модели
        loaded_models[exp_name] = {
            'actor': actor,
            'critic': critic,
            'config': config
        }
        
        # Получаем спецификации из конфигурации
        specs = make_specs(config.env.bounds)
        action_spec = specs["action"]
        observation_spec = specs["observation"]
        
        # Тестируем модели
        results = test_models_comprehensive(
            actor=actor,
            critic=critic,
            observation_spec=observation_spec,
            action_spec=action_spec,
            num_samples=num_test_samples,
            device=device
        )
        
        # Сохраняем результаты
        experiment_results[exp_name] = results
        
        print(f"✅ Эксперимент {exp_name} успешно протестирован")
        
    except Exception as e:
        print(f"❌ Ошибка при тестировании эксперимента {exp_name}: {str(e)}")
        continue

print(f"\n{'='*60}")
print(f"Завершено тестирование {len(experiment_results)} экспериментов")
print(f"{'='*60}")


## Визуализация результатов


In [ ]:
# Визуализируем результаты для каждого эксперимента
if experiment_results:
    for exp_name, results in experiment_results.items():
        print(f"\nВизуализация результатов для эксперимента: {exp_name}")
        plot_test_results(results, title=f"Результаты тестирования: {exp_name}")
else:
    print("Нет результатов для визуализации")


In [ ]:
# Сравниваем результаты всех экспериментов
if len(experiment_results) > 1:
    print("\nСравнение всех экспериментов:")
    compare_experiments(experiment_results)
else:
    print("Недостаточно экспериментов для сравнения")


## Тестирование с фиксированными входными данными


In [ ]:
# Тестируем модели на фиксированных входных данных
if loaded_models:
    print("Тестирование на фиксированных входных данных:")
    
    # Создаем фиксированные тестовые данные (соответствуют спецификациям реальной среды)
    # Наблюдения: [ADC_value, DAC_value, setpoint] в диапазоне [0, 10230] для ADC и setpoint, [0, 4095] для DAC
    fixed_observations = torch.tensor([
        [1000.0, 2000.0, 1200.0],  # Тестовое наблюдение 1
        [5000.0, 1000.0, 1200.0],  # Тестовое наблюдение 2
        [8000.0, 3000.0, 1200.0],  # Тестовое наблюдение 3
        [2000.0, 4000.0, 1200.0],  # Тестовое наблюдение 4
        [6000.0, 2000.0, 1200.0],  # Тестовое наблюдение 5
    ], device=device)
    
    # Действия: [Kp, Ki, Kd] в диапазоне [0, 17.5], [0, 55.0], [0, 0.01]
    fixed_actions = torch.tensor([
        [5.0, 20.0, 0.005],  # Тестовое действие 1
        [10.0, 30.0, 0.008], # Тестовое действие 2
        [15.0, 40.0, 0.010], # Тестовое действие 3
        [8.0, 25.0, 0.006],  # Тестовое действие 4
        [12.0, 35.0, 0.009], # Тестовое действие 5
    ], device=device)
    
    print(f"Фиксированные наблюдения:\n{fixed_observations}")
    print(f"Фиксированные действия:\n{fixed_actions}")
    
    # Тестируем каждую модель на фиксированных данных
    for exp_name, models in loaded_models.items():
        print(f"\n{'='*50}")
        print(f"Результаты для эксперимента: {exp_name}")
        print(f"{'='*50}")
        
        actor = models['actor']
        critic = models['critic']
        
        # Тестируем актора
        actor_output = test_actor(actor, fixed_observations, device)
        print(f"Действия актора на фиксированных наблюдениях:")
        print(actor_output)
        
        # Тестируем критика с действиями от актора
        critic_output_actor = test_critic(critic, fixed_observations, actor_output, device)
        print(f"Q-значения критика (действия от актора):")
        print(critic_output_actor)
        
        # Тестируем критика с фиксированными действиями
        critic_output_fixed = test_critic(critic, fixed_observations, fixed_actions, device)
        print(f"Q-значения критика (фиксированные действия):")
        print(critic_output_fixed)
        
        # Сравниваем результаты
        print(f"\nСравнение Q-значений:")
        print(f"Среднее Q-значение (действия от актора): {critic_output_actor.mean():.4f}")
        print(f"Среднее Q-значение (фиксированные действия): {critic_output_fixed.mean():.4f}")
        print(f"Разность: {critic_output_actor.mean() - critic_output_fixed.mean():.4f}")
        
else:
    print("Нет загруженных моделей для тестирования")


## Дополнительные функции для анализа


In [ ]:
def analyze_model_behavior(actor, critic, observation_spec, action_spec, 
                          num_samples=100, device="cpu"):
    """
    Анализирует поведение моделей на большом количестве образцов.
    
    Args:
        actor: Модель актора
        critic: Модель критика
        observation_spec: Спецификация наблюдений
        action_spec: Спецификация действий
        num_samples: Количество образцов для анализа
        device: Устройство для вычислений
    
    Returns:
        Словарь с результатами анализа
    """
    print(f"Анализ поведения моделей на {num_samples} образцах...")
    
    # Генерируем тестовые данные
    observations, random_actions = generate_test_data(observation_spec, action_spec, num_samples, device)
    
    # Получаем действия от актора
    actor_actions = test_actor(actor, observations, device)
    
    # Получаем Q-значения
    q_values_actor = test_critic(critic, observations, actor_actions, device)
    q_values_random = test_critic(critic, observations, random_actions, device)
    
    # Анализируем поведение
    analysis = {
        "actor_consistency": actor_actions.std(dim=0).mean().item(),
        "actor_range": (actor_actions.max(dim=0)[0] - actor_actions.min(dim=0)[0]).mean().item(),
        "q_value_advantage": (q_values_actor.mean() - q_values_random.mean()).item(),
        "q_value_consistency": q_values_actor.std().item(),
        "action_bounds_compliance": (
            (actor_actions >= action_spec.space.low).all() and 
            (actor_actions <= action_spec.space.high).all()
        )
    }
    
    print(f"Анализ завершен:")
    print(f"  Консистентность актора (среднее std): {analysis['actor_consistency']:.4f}")
    print(f"  Диапазон действий актора: {analysis['actor_range']:.4f}")
    print(f"  Преимущество Q-значений (актор vs случайные): {analysis['q_value_advantage']:.4f}")
    print(f"  Консистентность Q-значений: {analysis['q_value_consistency']:.4f}")
    print(f"  Соблюдение границ действий: {analysis['action_bounds_compliance']}")
    
    return analysis

# Анализируем поведение всех загруженных моделей
if loaded_models:
    print("Анализ поведения моделей:")
    behavior_analysis = {}
    
    for exp_name, models in loaded_models.items():
        print(f"\n{'='*50}")
        print(f"Анализ эксперимента: {exp_name}")
        print(f"{'='*50}")
        
        actor = models['actor']
        critic = models['critic']
        config = models['config']
        
        # Получаем спецификации из конфигурации
        specs = make_specs(config.env.bounds)
        action_spec = specs["action"]
        observation_spec = specs["observation"]
        
        # Анализируем поведение
        analysis = analyze_model_behavior(actor, critic, observation_spec, action_spec, 
                                        num_samples=100, device=device)
        behavior_analysis[exp_name] = analysis
    
    # Сравниваем анализ поведения
    print(f"\n{'='*60}")
    print("Сравнение анализа поведения:")
    print(f"{'='*60}")
    
    for metric in ["actor_consistency", "actor_range", "q_value_advantage", "q_value_consistency"]:
        print(f"\n{metric}:")
        for exp_name, analysis in behavior_analysis.items():
            print(f"  {exp_name}: {analysis[metric]:.4f}")
else:
    print("Нет загруженных моделей для анализа")
